In [1]:
import pandas as pd
import numpy as np

In [2]:
df=pd.read_csv("paysim_with_ml_scores.csv")
df.head()

,step,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud,balance_delta_org,balance_delta_dest,hour,type_CASH_OUT,type_DEBIT,type_PAYMENT,type_TRANSFER,ml_fraud_score,ml_anomaly_flag
0,278,330218.42,20866.00,351084.42,452419.57,122201.15,0,0,-330218.42,-330218.42,14,False,False,False,False,0.214322,0
1,15,11647.08,30370.00,18722.92,0.00,0.00,0,0,11647.08,0.00,15,False,False,True,False,0.007399,0
2,10,152264.21,106589.00,258853.21,201303.01,49038.80,0,0,-152264.21,-152264.21,10,False,False,False,False,0.124148,0
3,403,1551760.63,0.00,0.00,3198359.45,4750120.08,0,0,0.00,1551760.63,19,False,False,False,True,0.430251,0
4,206,78172.30,2921331.58,2999503.88,415821.90,337649.60,0,0,-78172.30,-78172.30,14,False,False,False,False,0.226824,0


In [3]:
ml_df = pd.read_csv("paysim_with_ml_scores.csv")
rules_df = pd.read_csv("paysim_with_rules.csv")

# Merge on the index
df = ml_df.merge(rules_df[["rule_score", "rule_reasons"]], left_index=True, right_index=True)

# Save combined dataset
df.to_csv(r"C:\Users\MAITHILI\Gen-AI_FraudGuard\paysim_ml_rules.csv", index=False)


In [4]:
df=pd.read_csv("paysim_ml_rules.csv")
df.head()

,step,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud,balance_delta_org,balance_delta_dest,hour,type_CASH_OUT,type_DEBIT,type_PAYMENT,type_TRANSFER,ml_fraud_score,ml_anomaly_flag,rule_score,rule_reasons
0,278,330218.42,20866.00,351084.42,452419.57,122201.15,0,0,-330218.42,-330218.42,14,False,False,False,False,0.214322,0,40,Balance mismatch detected;
1,15,11647.08,30370.00,18722.92,0.00,0.00,0,0,11647.08,0.00,15,False,False,True,False,0.007399,0,0,NaN
2,10,152264.21,106589.00,258853.21,201303.01,49038.80,0,0,-152264.21,-152264.21,10,False,False,False,False,0.124148,0,40,Balance mismatch detected;
3,403,1551760.63,0.00,0.00,3198359.45,4750120.08,0,0,0.00,1551760.63,19,False,False,False,True,0.430251,0,55,Balance mismatch detected; Transaction type is...
4,206,78172.30,2921331.58,2999503.88,415821.90,337649.60,0,0,-78172.30,-78172.30,14,False,False,False,False,0.226824,0,40,Balance mismatch detected;


In [5]:
df["rule_score_norm"] = df["rule_score"] / 100


In [6]:
# Add placeholder for LLM score
df["llm_score"] = 0  # you will replace this later with LLM reasoning

# Compute total fraud score
df["total_fraud_score"] = (
    0.6 * df["ml_fraud_score"] +
    0.2 * df["rule_score_norm"] +
    0.2 * df["llm_score"]
)


In [7]:
df["flagged_fraud"] = (df["total_fraud_score"] > 0.7).astype(int)


In [8]:
df[["ml_fraud_score", "rule_score", "rule_score_norm", "total_fraud_score", "flagged_fraud", "isFraud"]].sample(10)


,ml_fraud_score,rule_score,rule_score_norm,total_fraud_score,flagged_fraud,isFraud
45475,0.065205,0,0.00,0.039123,0,0
26241,0.251908,80,0.80,0.311145,0,0
72646,0.015821,55,0.55,0.119493,0,0
135868,0.399468,55,0.55,0.349681,0,0
23536,0.066133,55,0.55,0.149680,0,0
22178,0.084479,40,0.40,0.130687,0,0
31366,0.083951,55,0.55,0.160370,0,0
54558,0.037364,0,0.00,0.022418,0,0
47932,0.538404,80,0.80,0.483042,0,0
80067,0.158461,80,0.80,0.255076,0,0


In [9]:
df['flagged_fraud'].value_counts()

flagged_fraud
0    149927
1        73
Name: count, dtype: int64

In [10]:
df.to_csv(r"C:\Users\MAITHILI\Gen-AI_FraudGuard\paysim_fraud_scores.csv", index=False)
print("Saved final fraud scoring dataset")


Saved final fraud scoring dataset
